In [1]:
import numpy as np
import matplotlib.pyplot as plt
from database_methods import Table, to_show, head, process_data
import sqlite3
con = sqlite3.connect('database_results.db')
cursor = con.cursor()

In [2]:
mas = Table(connection=con, name='masters')
exp = Table(con, name = 'experiments')
meas = Table(con, name = 'measurements')

In [3]:
print(mas.columns)
print(exp.columns)
print(meas.columns)
print('-'*80)
mas.show_all()
exp.show_all()

CATEGORY, PERIOD, AMPLITUDE, MASTER, FWHM, R, S
DATE, MASTER_ID, DIP, FOLDER
EXPERIMENT_ID, NP, WAVELENGTH, FILE, AR_TH, AR_DIP, AR_NOISE, EXC_THS, RATE, INFO, COUNT, X, Y, CON, INP, INPPX, IBGPX, STD, stdX, stdY, stdCON, stdINP, stdINPPX, stdIBGPX, stdSTD, NOTES, AR_MIN, AR_COND, EXP_TYPE, OPT
--------------------------------------------------------------------------------
ID	CATEGORY	PERIOD	AMPLITUDE	MASTER	FWHM	R	S
1	600	405	45	M5	23.0	8.0	15.9
2	600	405	53	P3	25.0	19.7	3.8
3	650	452	32	K4	12.5	11.9	17.8
4	650	452	40	K5	15.0	18.0	8.8
5	750	528	24	L3	7.0	13.8	78.5
6	750	526	30	Q4	9.5	22.0	30.7
ID	DATE	MASTER_ID	DIP	FOLDER
1	20-02-25	2	615	20_02_25_P3/
2	20-02-26	6	727	20_02_26_Q3/
3	20-03-16	4	658	20_03_16_K5/
5	20-02-25	1	598	20_02_25_M5/
6	20-03-16	3	647	20_03_16_K4/
7	20-04-15	5	740	20_04_15_L3/
8	20-04-14	1	606	20_04_14_M5/
9	20-04-20	6	729	20_04_20_Q4/


In [4]:
con.commit()

In [6]:
# 
columns = 'MASTER, S, AVG(DIP), AVG(INP), AVG(INP/IBGPX), AVG(CON), AVG(INPPX), AVG(IBGPX)'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID
WHERE NP = 80 AND OPT = 1
GROUP BY MASTER
ORDER BY DIP DESC
;
   """.format(columns))

head(columns)

for row in data:
    print(str(to_show(row)).replace(', ', '\t').replace("'", "")[1:-1])


MASTER	S	AVG(DIP	AVG(INP	AVG(INP	AVG(CON	AVG(INP	AVG(IBG
L3	78.50	740.00	19.64	68.61	3.11	0.9066	0.2933
Q4	30.70	729.00	26.32	93.35	10.18	2.85	0.2820
K5	8.80	658.00	15.64	55.57	5.17	1.44	0.2827
K4	17.80	647.00	9.48	36.03	4.27	1.15	0.2686
P3	3.80	615.00	4.90	23.55	7.52	1.53	0.2082
M5	15.90	606.00	2.51	10.67	5.97	1.41	0.2375


In [51]:
# 
columns = 'NP, FILE, X, Y, CON, X*Y, INP, stdINP INPPX, IBGPX, EXP_TYPE'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID
WHERE MASTER = 'Q4' AND DATE='20-04-20' AND WAVELENGTH = 732 AND NP = 80

ORDER BY NP DESC
;
   """.format(columns))

head(columns)

i=0
for row in data:
    print(str(to_show(row)).replace(', ', '\t').replace("'", "")[1:-1])
    i+=1
    
    
print('---\n{}'.format(i))

NP	FILE	X	Y	CON	X*Y	INP	stdINP 	IBGPX	EXP_TYP
80	raw_02_	24.29	9.28	11.33	225.36	25.61	19.26	0.2904	-
80	raw_17_	25.15	10.14	8.20	254.92	23.66	22.90	0.2759	strep
80	raw_15_	27.30	9.62	11.00	262.61	29.68	22.92	0.2798	sh
---
3


In [54]:
# 
columns = 'NP, FILE, X, Y, CON, X*Y, INP, stdINP INPPX, IBGPX, EXP_TYPE'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID
WHERE MASTER = 'P3' AND WAVELENGTH = 612 AND NP = 80

ORDER BY NP DESC
;
   """.format(columns))

head(columns)

i=0
for row in data:
    print(str(to_show(row)).replace(', ', '\t').replace("'", "")[1:-1])
    i+=1
    
    
print('---\n{}'.format(i))

NP	FILE	X	Y	CON	X*Y	INP	stdINP 	IBGPX	EXP_TYP
80	raw_10_	5.06	5.98	7.86	30.26	4.53	2.27	0.1912	-
80	raw_29_	5.11	6.36	8.16	32.51	4.91	2.18	0.2032	sh
80	raw_32_	5.11	6.11	7.43	31.24	4.54	2.02	0.2021	strep
---
3


In [102]:
# 
columns = 'MEAS.ID, MASTER, NP, WAVELENGTH, FOLDER, FILE, RATE, X, Y, CON, X*Y, INP, stdINP INPPX, IBGPX, EXP_TYPE, OPT'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID

ORDER BY RATE DESC
;
   """.format(columns))

head(columns)

i=0
for row in data:
    print(str(to_show(row)).replace(', ', '\t').replace("'", "")[1:-1])
    i+=1
    
    
print('---\n{}'.format(i))

MEAS.ID	MASTER	NP	WAVELEN	FOLDER	FILE	RATE	X	Y	CON	X*Y	INP	stdINP 	IBGPX	EXP_TYP	OPT
50	P3	80	612	20_02_2	raw_32_	168.45	5.11	6.11	7.43	31.24	4.54	2.02	0.2021	strep	1
46	P3	80	606	20_02_2	raw_13_	101.54	5.33	6.04	6.32	32.18	5.42	2.60	0.2714	-	0
8	Q4	60	732	20_02_2	raw_18_	97.04	27.16	12.88	3.97	350.01	31.03	19.98	0.5038	-	0
9	Q4	60	732	20_02_2	raw_21_	96.97	28.07	13.13	3.82	368.74	30.71	22.48	0.4713	sh	0
7	Q4	80	724	20_02_2	raw_09_	89.81	40.87	11.16	3.05	456.27	20.65	15.06	0.3386	-	0
2	Q4	80	732	20_02_2	raw_04_	85.33	32.05	13.17	4.42	422.06	46.00	33.63	0.5202	-	0
45	P3	80	608	20_02_2	raw_12_	80.47	5.34	6.10	6.25	32.59	5.07	2.49	0.2577	-	0
6	Q4	80	726	20_02_2	raw_08_	70.91	41.31	13.22	3.64	546.22	20.36	12.73	0.2148	-	0
3	Q4	80	734	20_02_2	raw_05_	70.29	24.95	12.71	5.38	317.05	36.50	21.71	0.5129	-	0
71	M5	80	628	20_04_1	raw_08_	68.20	3.74	3.30	5.41	12.37	1.97	1.43	0.2508	-	1
47	P3	80	602	20_02_2	raw_14_	65.62	4.93	5.89	6.51	28.99	6.21	2.97	0.3352	-	0
86	Q4	80	732	20_04_2	raw_17_	64.45	25

In [66]:
exp.show_all()

ID	DATE	MASTER_ID	DIP	FOLDER
1	20-02-25	2	615	20_02_25_P3/
2	20-02-26	6	727	20_02_26_Q3/
3	20-03-16	4	658	20_03_16_K5/
5	20-02-25	1	598	20_02_25_M5/
6	20-03-16	3	647	20_03_16_K4/
7	20-04-15	5	740	20_04_15_L3/
8	20-04-14	1	606	20_04_14_M5/
9	20-04-20	6	729	20_04_20_Q4/


In [96]:
con.commit()

In [12]:
folder = '20_04_20_Q4/'
file = 'raw_17_1'

columns = 'COUNT(MEAS.ID), MEAS.ID'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID 
WHERE FOLDER = '{}' AND FILE = '{}'
;
   """.format(columns, folder, file)
                  )

In [26]:
[d for d in data][0]

(3, 79)

In [20]:
b

In [28]:
columns = 'NP, WAVELENGTH, AR_TH, AR_NOISE, EXC_THS, AR_MIN, AR_COND, exp_type'
data = con.execute("""
SELECT {} FROM 'measurements'
WHERE WAVELENGTH=732
ORDER BY WAVELENGTH ASC;
   """.format(columns))

print(columns.replace(', ','\t'))
for row in data:
    print(str(row).replace(', ', '\t').replace("'", "")[1:-1])

NP	WAVELENGTH	AR_TH	AR_NOISE	EXC_THS	AR_MIN	AR_COND	exp_type
80	732	60.0	0.001	[2]	10	2	-
60	732	59.0	0.001	[2]	10	2	-
60	732	46.0	0.001	[2]	10	2	sh
40	732	12.0	0.001	[1.1]	2	2	-
80	732	100.0	0.001	[2]	2	2	-
60	732	20.0	0.001	[2]	2	2	-
80	732	40.0	0.001	[2]	2	2	strep
147	732	8.0	0.001	[2]	2	2	ps
40	732	9.0	0.001	[2]	6	2	-
80	732	31.0	0.001	[2]	6	2	sh


In [59]:
con.commit()

In [51]:
con.close()